# Sending and receiving tasks and data

In the previous notebook, we explored how to build custom components, in particular `Controller`s and `Executor`s and have them do some basic logging.

Now, we'll show how to make them interact with each other and execute workflows by implementing their `control_flow` and `execute` method respectively.


## Execution/communication primitives
There are 2 main elements we haven't yet seen that we can use to execute workflows: `Task`s and data-exchange primitives (`Shareable`s and `DXO`s).

- A `Task` represents a unit of work assigned by the controller to the executors. It includes:
    - `name`: The name of the task.
    - `data`: The actual data or instructions associated with the task provided as a `Shareable`.
    - Additional properties or metadata.

    How is it used?: The Controller creates tasks and sends them to the clients (we'll dive into how in a bit). Executors receive these tasks and process them in the execute method.

- `Shareable` and `DXO` (Data Exchange Object) are two key data structures used for data-exchange between parties in NVFlare. They serve different purposes but are designed to interact closely with each other.

    - A `Shareable` is a flexible data container used for transmitting data between different components in NVFlare. It's essentially a Python dictionary (dict) that can hold any serializable data. It can contain both data and metadata and is used as the primary medium for sending and receiving messages over the network. As said, values must be serializable to ensure they can be transmitted between processes or over the network.

    - A `DXO` provides a standardized way to represent and handle common types of data exchanged during federated learning, such as model weights, gradients, metrics, and hyperparameters. It encapsulates the actual data along with metadata about the data and defines the kind of data being exchanged using `DataKind`. `DXO`s help in maintaining consistency and understanding of data across different components.

    - `Shareable`s and `DXO`s can be easily converted into each other by using the `to_shareable()` and `from_shareable()` methods. We usually use a `DXO` to to package the data and metadata, and then convert it to a `Shareable` to send it over the network. On the receiving end, we convert the `Shareable` back to a `DXO` to access the data and metadata directly.

## Sending and receiving tasks

To get an initial idea of how `Task`s work, let's create a custom controller that simply creates a `Task` with name `"hello"` and no attached data, and broadcasts it to all clients.
We will use the `broadcast_and_wait` method to broadcast the task to all clients and wait for all to respond.

```python
from nvflare.apis.fl_context import FLContext
from nvflare.apis.impl.controller import Controller
from nvflare.apis.signal import Signal
from nvflare.apis.controller_spec import Task
from nvflare.apis.shareable import Shareable

class HelloController(Controller):

    def control_flow(self, abort_signal: Signal, fl_ctx: FLContext):        
        # Create the task with name "hello"
        task = Task(name="hello", data=Shareable())

        # Broadcast the task to all clients and wait for all to respond
        self.broadcast_and_wait(
            task=task,
            targets=None, # meaning all clients, determined dynamically
            min_responses=0, # meaning all clients
            fl_ctx=fl_ctx,
        )
```

Now, the question would be: how do we receive this task in the executor? As anticipated in the previous notebook, we can do this by implementing the `execute` method in the executor. This method is called by the NVFlare framework when a task is received by the executor. Let's do that and just log the name of the received task.

```python
from nvflare.apis.executor import Executor
from nvflare.apis.shareable import make_reply
from nvflare.apis.fl_constant import ReturnCode


class HelloExecutor(Executor):

    def execute(
        self,
        task_name: str,
        shareable: Shareable,
        fl_ctx: FLContext,
        abort_signal: Signal,
    ):
        if task_name == "say_hello":
            self.log_info(fl_ctx, f"Received task with name {task_name} and data {shareable}")
            return make_reply(ReturnCode.OK)
```

Now, let's run the job and see what happens. As usual, we're putting the implementation of our custom components in the `modules.py` file, so that we can run the job via the NVFlare simulator. Feel free to implement the `modules.py` yourself or use the provided one.

> HINT: If you go on with implementing the controller and executor yourself in `modules.py`, make sure to implement all the required abstract methods as we discussed in the previous notebook. 

In [1]:
from nvflare.job_config.api import FedJob
from modules import HelloController, HelloExecutor

job = FedJob(name="hello_job")

controller = HelloController()
job.to_server(controller)

num_clients = 3
for i in range(num_clients):
    executor = HelloExecutor()
    job.to(executor, f"site-{i}")

job.simulator_run("./tmp/")

2025-02-06 18:16:08,493 - SimulatorRunner - INFO - Create the Simulator Server.
2025-02-06 18:16:08,494 - CoreCell - INFO - server: creating listener on tcp://0:54139
2025-02-06 18:16:08,522 - CoreCell - INFO - server: created backbone external listener for tcp://0:54139
2025-02-06 18:16:08,522 - ConnectorManager - INFO - 58964: Try start_listener Listener resources: {'secure': False, 'host': 'localhost', <DriverParams.CONNECTION_SECURITY: 'connection_security'>: 'clear'}
2025-02-06 18:16:08,523 - conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:43224] is starting
2025-02-06 18:16:09,024 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:43224
2025-02-06 18:16:09,024 - conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:54139] is starting
2025-02-06 18:16:09,025 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Request
2025-02-06 18:16:09,025 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Abort
2025-0

By inspecting the logs, you should see that all clients are receiving the `"hello"` task and logging something like

```
Received task with name hello and data {'__headers__': {'__task_name__': 'hello', '__task_id__': '6a007af8-823f-49ec-a685-71162a499405', '__cookie_jar__': {'__workflow__': 'controller', '__task_id__': '6a007af8-823f-49ec-a685-71162a499405'}, '__audit_event_id__': None, '__wait_time__': 2, 'task_name': 'hello', '__peer_ctx__': <nvflare.apis.fl_context.FLContext object at 0x12113e9f0>, '__peer_props__': {'__run_num__': 'simulate_job', '__identity_name__': 'simulator_server'}}}
```

The object you see logged is the `Shareable` object that was sent by the controller.

> Exercise: as an exercise, feel free to experiment with other methods for sending messages, like `broadcast`, `send` or `send_and_wait` or send messages only to specific clients.

> HINT: by default clients are named `"site-1"`, `"site-2"`, ...

## Adding data to tasks and receiving responses

Now that we know how to send tasks, let's add some data to the task and see how we can receive it in the executor. As already mentioned, we'll use `DXO`s and `Shareable`s to to that.

We'll use a `DXO` to package the data and metadata, and then convert it to a `Shareable` to send it over the network. On the receiving end, we convert the `Shareable` back to a `DXO` to access the data and metadata directly.

Let's start with the controller and include a message as a data.

```python
from nvflare.apis.dxo import DXO, DataKind, from_shareable

class HelloDataController(Controller):

    def control_flow(self, abort_signal: Signal, fl_ctx: FLContext):        
        # Prepare data to send to the clients
        data = DXO(
            data_kind=DataKind.APP_DEFINED,
            data={"message": "howdy, I'm the controller"},
        ).to_shareable()

        # Create the task 
        task = Task(name="say_hello", data=data)

        # Broadcast the task to all clients and wait for all to respond
        self.broadcast_and_wait(
            task=task,
            targets=None, # meaning all clients
            min_responses=0,
            fl_ctx=fl_ctx,
        )
```

Now, on the executor side, let's also convert the `Shareable` back to a `DXO` so that we can easily access its the `data` field.

```python
from nvflare.apis.dxo import from_shareable

class HelloDataExecutor(Executor):

    def execute(
        self,
        task_name: str,
        shareable: Shareable,
        fl_ctx: FLContext,
        abort_signal: Signal,
    ):
        if task_name == "say_hello":
            received_dxo = from_shareable(shareable)
            message = received_dxo.data["message"]
            self.log_info(fl_ctx, f"Received message from server: {message}")
            return make_reply(ReturnCode.OK)
```

Let's see if everything works as expected.

In [2]:
from modules import HelloDataController, HelloDataExecutor

job = FedJob(name="hello_data_job")

controller = HelloDataController()
job.to_server(controller)

num_clients = 3
for i in range(num_clients):
    executor = HelloDataExecutor()
    job.to(executor, f"site-{i}")

job.simulator_run("./tmp/")

2025-02-06 18:16:20,349 - SimulatorRunner - INFO - Create the Simulator Server.
2025-02-06 18:16:20,350 - CoreCell - INFO - server: creating listener on tcp://0:54307
2025-02-06 18:16:20,376 - CoreCell - INFO - server: created backbone external listener for tcp://0:54307
2025-02-06 18:16:20,376 - ConnectorManager - INFO - 59029: Try start_listener Listener resources: {'secure': False, 'host': 'localhost', <DriverParams.CONNECTION_SECURITY: 'connection_security'>: 'clear'}
2025-02-06 18:16:20,377 - conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:60608] is starting
2025-02-06 18:16:20,879 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:60608
2025-02-06 18:16:20,879 - conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:54307] is starting
2025-02-06 18:16:20,880 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Request
2025-02-06 18:16:20,880 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Abort
2025-0

Now, looking at the logs above, you should be able to to see that the clients are receiving the message from the server and logging 

```
Received message from server: howdy, I'm the controller
```

### Responding

We have just seen how the server can send tasks and data to the clients, but what happens to the `Shareable` the clients are sending back to the server as a response?

Let's assume our executor is sending back a message saying hello back to the server.

```python
class HelloResponseExecutor(Executor):

    def execute(
        self,
        task_name: str,
        shareable: Shareable,
        fl_ctx: FLContext,
        abort_signal: Signal,
    ):
        if task_name == "hello":
            received_dxo = from_shareable(shareable)
            message = received_dxo.data["message"]
            self.log_info(fl_ctx, f"Received message: {message}")
            self.log_info(fl_ctx, "Sending response to server...")
            response = DXO(
                data_kind=DataKind.APP_DEFINED,
                data={"message": "howdy, I'm a client"},
            ).to_shareable()
            return response
```

How can we let the server access that information? 

To do that we need to add a callback to the task specifying how received responses are handled. Let's see how we can do that.

```python
class HelloResponseController(Controller):

    def control_flow(self, abort_signal: Signal, fl_ctx: FLContext):        
        data = DXO(
            data_kind=DataKind.APP_DEFINED,
            data={"message": "howdy, I'm the controller"},
        ).to_shareable()

        # add callback to the task
        task = Task(name="hello", data=shareable, result_received_cb=self._process_client_response)

        self.broadcast_and_wait(
            task=task,
            targets=None, 
            min_responses=0,
            fl_ctx=fl_ctx,
        )
    
    def start_controller(self, fl_ctx: FLContext):
        self.log_info(fl_ctx, "Starting the controller...")

    def stop_controller(self, fl_ctx: FLContext):
        self.log_info(fl_ctx, "Stopping the controller...")

    # implement callback function
    def _process_client_response(self, client_task, fl_ctx: FLContext):
        task = client_task.task
        client = client_task.client
        response = client_task.result
        received_msg = from_shareable(response).data["message"]

        self.log_info(fl_ctx, f"Received message {received_msg} from client {client.name} for task {task.name}")
```

Let's see that in action - you should now see messages like `Received message howdy, I'm a client from client site-0 for task hello` in the server logs.

In [3]:
from modules import HelloResponseController, HelloResponseExecutor

job = FedJob(name="response_job")

controller = HelloResponseController()
job.to_server(controller)

num_clients = 3
for i in range(num_clients):
    executor = HelloResponseExecutor()
    job.to(executor, f"site-{i}")

job.simulator_run("./tmp/")

2025-02-06 18:16:32,201 - SimulatorRunner - INFO - Create the Simulator Server.
2025-02-06 18:16:32,202 - CoreCell - INFO - server: creating listener on tcp://0:54469
2025-02-06 18:16:32,228 - CoreCell - INFO - server: created backbone external listener for tcp://0:54469
2025-02-06 18:16:32,228 - ConnectorManager - INFO - 59088: Try start_listener Listener resources: {'secure': False, 'host': 'localhost', <DriverParams.CONNECTION_SECURITY: 'connection_security'>: 'clear'}
2025-02-06 18:16:32,229 - conn_manager - INFO - Connector [CH00002 PASSIVE tcp://0:6851] is starting
2025-02-06 18:16:32,730 - CoreCell - INFO - server: created backbone internal listener for tcp://localhost:6851
2025-02-06 18:16:32,730 - conn_manager - INFO - Connector [CH00001 PASSIVE tcp://0:54469] is starting
2025-02-06 18:16:32,731 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Request
2025-02-06 18:16:32,731 - AuxRunner - INFO - registered aux handler for topic ObjectStreamer.Abort
2025-02-

## Exercise

As an exercise try to modify the `HelloResponseController` to include it's name in the response it's sending to the server. Try to retrieve that when needed or to set it when the client starts to run.

> HINT 1: you can access the client's name through the `get_identity_name` method of the `FLContext`.

> HINT 2: remember from the previous notebook that you can use the `handle_event` method to perform actions when certain events happen (for example `EventType.START_RUN`)

> HINT 3: if you get stuck look at `nvflare.app_opt.p2p.executors.base_dit_opt_executor.py` for inspiration.